# Fatigue Detection Using Facial Landmarks and ANN

## Input : coordinates (x,y) of face landmarks 
## Output : Fatigue level


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

import os
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### Import data analyse class for cacluls parameter from landamrks

In [2]:
from data_analyse import AnalyseData
from format_data import DataFormator

### Define Path of csv data and video infos

In [3]:
video_name = "DESFAM_Semaine 2-Vendredi_PVT_H64"
video_infos_path = "data/stage_data_out/videos_infos.csv"
video_path = "data/stage_data_out/DESFAM_Semaine 2-Vendredi_PVT_H64_hog.csv"

### Append Measure

In [4]:
analyse_data = AnalyseData(video_path)
analyse_data.measure_ear()

,frame,ear_l,ear_r,ear
0,0,0.387097,0.415926,0.401511
1,1,0.403226,0.385297,0.394261
2,2,0.401164,0.399806,0.400485
3,3,0.387566,0.399806,0.393686
4,4,0.355385,0.399806,0.377596
...,...,...,...,...
12904,30431,0.316491,0.316491,0.316491
12905,30432,0.338534,0.354654,0.346594
12906,30433,0.334254,0.333979,0.334116
12907,30434,0.322413,0.339441,0.330927


### Append Label

In [5]:
df_ear = DataFormator.make_label_df(num_min = 2, video_name = video_name, df_measure= analyse_data.df_measure)


In [6]:
df_temporal, df_label = DataFormator.make_df_temporal_label([10],df_ear)

ure : ear_l
window : 10
frame : 30271.0
measure : ear_r
window : 10
frame : 30271.0
measure : ear
window : 10
frame : 30272.0
measure : ear_l
window : 10
frame : 30272.0
measure : ear_r
window : 10
frame : 30272.0
measure : ear
window : 10
frame : 30273.0
measure : ear_l
window : 10
frame : 30273.0
measure : ear_r
window : 10
frame : 30273.0
measure : ear
window : 10
frame : 30274.0
measure : ear_l
window : 10
frame : 30274.0
measure : ear_r
window : 10
frame : 30274.0
measure : ear
window : 10
frame : 30275.0
measure : ear_l
window : 10
frame : 30275.0
measure : ear_r
window : 10
frame : 30275.0
measure : ear
window : 10
frame : 30276.0
measure : ear_l
window : 10
frame : 30276.0
measure : ear_r
window : 10
frame : 30276.0
measure : ear
window : 10
frame : 30277.0
measure : ear_l
window : 10
frame : 30277.0
measure : ear_r
window : 10
frame : 30277.0
measure : ear
window : 10
frame : 30278.0
measure : ear_l
window : 10
frame : 30278.0
measure : ear_r
window : 10
frame : 30278.0
measur

,frame_10,ear_l_10,ear_r_10,ear_10,Target_10
0.0,NaN,"[0.3870967741935484, 0.4032258064516129, 0.401...","[0.415925535378257, 0.38529677944076135, 0.399...","[0.4015111547859027, 0.3942612929461871, 0.400...",NaN
1.0,NaN,"[0.4032258064516129, 0.40116367912272516, 0.38...","[0.38529677944076135, 0.39980577069874773, 0.3...","[0.3942612929461871, 0.40048472491073644, 0.39...",NaN
2.0,NaN,"[0.40116367912272516, 0.3875660596535488, 0.35...","[0.39980577069874773, 0.39980577069874773, 0.3...","[0.40048472491073644, 0.39368591517614826, 0.3...",NaN
3.0,NaN,"[0.3875660596535488, 0.35538548306642387, 0.38...","[0.39980577069874773, 0.39980577069874773, 0.3...","[0.39368591517614826, 0.3775956268825858, 0.38...",NaN
4.0,NaN,"[0.35538548306642387, 0.3875660596535488, 0.38...","[0.39980577069874773, 0.3875660596535488, 0.38...","[0.3775956268825858, 0.3875660596535488, 0.387...",NaN
...,...,...,...,...,...
30420.0,NaN,"[0.33342447186042473, 0.3342538911019798, 0.34...","[0.349805717442704, 0.3333333333333333, 0.3333...","[0.34161509465156437, 0.3337936122176566, 0.33...",NaN
30421.0,NaN,"[0.3342538911019798, 0.3457262316037574, 0.332...","[0.3333333333333333, 0.3333333333333333, 0.333...","[0.3337936122176566, 0.33952978246854537, 0.33...",NaN
30422.0,NaN,"[0.3457262316037574, 0.3325950526188697, 0.333...","[0.3333333333333333, 0.3333333333333333, 0.350...","[0.33952978246854537, 0.33296419297610147, 0.3...",NaN
30423.0,NaN,"[0.3325950526188697, 0.33342447186042473, 0.34...","[0.3333333333333333, 0.35075601695312103, 0.35...","[0.33296419297610147, 0.3420902444067729, 0.34...",NaN


In [4]:
df_tab = DataFormator.make_df_feature(df_temporal, df_label, [10])

['ear_l_10', 'ear_r_10', 'ear_10']


In [230]:
for df in df_tab : 
    DataFormator.save_df(df, video_name)